# VA DEQ Site Join

The purpose of this notebook is to combine the old and the new embeddedness datasets provided by the Virginia Department of Environmental Quality. The new dataset (ProbMon) contains many pre-calculated attributes for each site. The old dataset (TMDLsummary) contains attributes needed to calculate shear stress. The final dataset will be exported to Excel then analyzed for correlations in a separate Jnotebook.

E. Reilly Oare

In [4]:
# Standard Libraries
import os  # File handling and directory management

# Data Handling
import pandas as pd  # Data manipulation and analysis

### Step 1. Import datasets

In [5]:
# Define base directory
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), "../../.."))

In [6]:
# Read in old excel file
old_emd_path = os.path.join(BASE_DIR, "data", "raw", "virginia", "TMDLsummary_2023-06-01.xlsx")
oldemd = pd.read_excel(old_emd_path, header = 0)

# Read in new excel file (has coordinates attached to site)
new_emd_path = os.path.join(BASE_DIR, "data", "raw", "virginia", "Probmon2001-2022.xlsx")
newemd = pd.read_excel(new_emd_path, header = 0)

### Step 2. Check for null values

In [7]:
# Since our target feature is embeddedness, we will search for null values in that column only
print("The number of null values in the old dataset is", oldemd['Xembed'].isnull().sum().sum())
print("The number of null values in the new dataset is", newemd['Embed_PCT'].isnull().sum().sum())

The number of null values in the old dataset is 0
The number of null values in the new dataset is 243


*For now, we will need to delete the 243 datapoints that do not have embeddedness since that is our target.*

In [8]:
# Delete all data points in new dataset (ProbMon) that don't have embeddedness measurements
newemd.dropna(subset = 'Embed_PCT', inplace = True)

print("The number of null values in the new dataset is now", newemd['Embed_PCT'].isnull().sum().sum())

The number of null values in the new dataset is now 0


### Step 2. Rename datasets for joining

In [9]:
# Rename columns in newemd to match oldemd
newemd = newemd.rename(columns={'Embed_PCT':'Xembed',
                      'BL_CB_GR_Embed_PCT':'BL_CB_GR_transectPCT'
                      })

In [10]:
# Create Year column in old dataset for indexing
oldemd['Year'] = oldemd['Date'].dt.year

### Step 3. Drop uneccessary columns

In [11]:
# Tables were opened in Excel to identify columns of interest

# Drop columns in oldemd
oldemd.drop(columns=['SampleID',
                     'Interval',
                     'SiteFlag',
                     'ThalwegN'], inplace = True)

# Drop columns in newemd
newemd.drop(columns=['DataSource',
                     'StationID_Trend',
                     'stratum',
                     'designweight',
                     'weightcategory',
                     'station',
                     'state',
                     'status',
                     'comment',
                     'set',
                     'Order',
                     'BasinSize',
                     'StreamSizeCatPhase',
                     'IR2008',
                     'IR2010',
                     'IR2012',
                     'IR2014',
                     'IR2016',
                     'IR2018',
                     'IR2020',
                     'IR2022',
                     'IR2024',
                     'Hg-C',
                     'YearSampled',
                     'NLCD',
                     'MunMajor',
                     'MunMinor',
                     'IndMajor',
                     'IndMinor',
                    ], 
            inplace = True)

### Step 4. Join Tables

In [12]:
# Use a left join
vadeq_emd = oldemd.merge(newemd, on = ['StationID',
                                       'Year',
                                       'Xembed'],
                         how = "left")

In [13]:
# Display joined dataframe
vadeq_emd

,StationID,Date,ReachLength,Slope_x,RP100,BR_PCT,HP_PCT,RC_PCT,BL_PCT,CB_PCT,...,POPDENS2020,POPCHG2000_2010,POPCHG2010_2020,POPCHG2000_2020,RDLEN,RDLEN120,STXRD_CNT,RDDENS,pctRoadLengthInRiparian,STXRD
0,1AACO006.10,2006-11-21,440,0.220,34.752500,7.692308,6.730769,0.000000,9.615385,14.423077,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1AACO004.84,2008-06-25,320,0.521,25.757012,0.000000,0.952381,0.000000,5.714286,27.619048,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1AACO006.10,2008-06-26,520,0.173,35.429907,2.857143,7.619048,0.952381,4.761905,21.904762,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1AACO009.14,2008-06-26,560,0.223,22.451871,2.857143,4.761905,0.000000,9.523810,18.095238,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1AAUA017.60,2005-09-22,160,0.400,19.910507,0.000000,0.000000,0.000000,7.619048,34.285714,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1120,2BXRK001.64,2022-10-26,150,1.287,9.486085,0.952381,5.714286,0.952381,0.000000,13.333333,...,6.680674,-16.237075,-3.340934,-19.035539,5757.591719,1574.982250,2.0,2.716352,27.354879,0.894433
1121,2-JKS070.06,2022-09-12,800,0.450,25.065800,9.523810,0.000000,0.000000,5.714286,30.476190,...,2.198019,5.420818,-8.892221,-3.953434,451582.048000,86994.700400,69.0,4.955175,19.264428,0.960441
1122,4ASNA007.82,2022-09-20,320,0.430,15.910317,0.000000,0.000000,0.000000,0.000000,0.000000,...,13.829188,4.511600,-6.086899,-1.849916,617340.395400,53983.910780,90.0,1.004842,8.744594,0.451753
1123,8-MIC001.47,2022-10-25,150,0.300,10.014865,0.000000,9.708738,0.000000,0.000000,0.000000,...,35.564565,-7.758684,5.915693,-2.301971,58550.497110,7208.478259,12.0,1.883880,12.311558,0.815454


In [14]:
# Push to processed data folder for use in EDA, etc.
unclean_dir = os.path.join(BASE_DIR, "data", "processed", "uncleaned_vadeq_emd_joined.xlsx")
vadeq_emd.to_excel(unclean_dir)

### Step 4. Clean up VA DEQ dataset

In [15]:
# Check dataset for null values
percent_missing = vadeq_emd.isnull().sum()*100/len(vadeq_emd) # Calculates percent missing
missing_value_df = pd.DataFrame({'column_name':vadeq_emd.columns,
                                'percent_missing':percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True)

# Display dataframe
missing_value_df.sort_values(by = 'percent_missing',
                            ascending = False)

,column_name,percent_missing
THALLIUMppm,THALLIUMppm,96.177778
NICKELppm,NICKELppm,96.177778
ARSENICppm,ARSENICppm,96.177778
SELENIUMppm,SELENIUMppm,96.177778
ALUMINUMppm,ALUMINUMppm,96.177778
...,...,...
RP100,RP100,0.000000
Slope_x,Slope_x,0.000000
Date,Date,0.000000
ReachLength,ReachLength,0.000000


In [16]:
# Create a list of columns that have >80% of values missing
missing_cols = []
for index, row in missing_value_df.iterrows():
    if row['percent_missing'] > 80:
        missing_cols.append(row['column_name'])

missing_cols    

['RNAT30',
 'RHUM1',
 'RTotBAR1',
 'RURB1',
 'RMBAR1',
 'RAGT1',
 'RAGP1',
 'RAGC1',
 'RFOR30',
 'RMBAR30',
 'RSHRB30',
 'RNG30',
 'RBAR30',
 'RTotBAR30',
 'RHUM30',
 'RURB30',
 'RAGT30',
 'RBAR1',
 'RWETL30',
 'RNG1',
 'PBAR',
 'RWETL1',
 'N_INDEX',
 'PFOR',
 'PWETL',
 'PSHRB',
 'PNG',
 'RAGP30',
 'PTotBAR',
 'U_INDEX',
 'PURB',
 'PMBAR',
 'PAGT',
 'PAGP',
 'PAGC',
 'S',
 'H',
 'Hprime',
 'C',
 'RNAT1',
 'RFOR1',
 'RSHRB1',
 'RAGC30',
 'damcount',
 'RFOR120',
 'SLPMEAN',
 'SLPSD',
 'SLPRANGE',
 'wshdRain_mmyr',
 'siteRain_mmyr',
 'wshdRain_inyr',
 'siteRain_inyr',
 'wshdPOP2000',
 'POPDENS2000',
 'wshdPOP2010',
 'POPDENS2010',
 'wshdPOP2020',
 'POPDENS2020',
 'POPCHG2000_2010',
 'POPCHG2000_2020',
 'RDLEN',
 'RDLEN120',
 'STXRD_CNT',
 'RDDENS',
 'SLPMAX',
 'RNAT120',
 'SLPMIN',
 'ELEVSD',
 'RWETL120',
 'RSHRB120',
 'RNG120',
 'RBAR120',
 'RTotBAR120',
 'RHUM120',
 'RURB120',
 'RMBAR120',
 'RAGT120',
 'RAGP120',
 'RAGC120',
 'sqMileImp',
 'wshdImpPCT',
 'PWater',
 'STRMLEN',
 'STRMDENS

In [17]:
# Remove columns from vdeq_emd that are missing more than 80% of their data
for i in missing_cols:
    if i in vadeq_emd.columns:
        vadeq_emd.drop(columns=i, inplace=True)

# Display trimmed dataframe        
vadeq_emd

,StationID,Date,ReachLength,Slope_x,RP100,BR_PCT,HP_PCT,RC_PCT,BL_PCT,CB_PCT,...,Xwid,XBKF_W,BKF_depth_in_meters,BKFW_BKFD,incised_depth,Xembed,BL_CB_GR_transectPCT_x,BL_CB_GRmeanEmbed,LRBS2,Year
0,1AACO006.10,2006-11-21,440,0.220,34.752500,7.692308,6.730769,0.000000,9.615385,14.423077,...,14.690476,18.636364,1.123018,16.594890,2.547564,62.545455,66.666667,52.500000,0.517308,2006
1,1AACO004.84,2008-06-25,320,0.521,25.757012,0.000000,0.952381,0.000000,5.714286,27.619048,...,9.580952,15.372727,0.989473,15.536282,2.275836,51.636364,69.090909,37.894737,-0.245939,2008
2,1AACO006.10,2008-06-26,520,0.173,35.429907,2.857143,7.619048,0.952381,4.761905,21.904762,...,13.819048,17.800000,1.112973,15.993204,2.990245,56.818182,67.272727,49.594595,0.374672,2008
3,1AACO009.14,2008-06-26,560,0.223,22.451871,2.857143,4.761905,0.000000,9.523810,18.095238,...,12.790476,15.681818,0.857991,18.277371,2.221627,63.454545,60.000000,59.696970,0.459602,2008
4,1AAUA017.60,2005-09-22,160,0.400,19.910507,0.000000,0.000000,0.000000,7.619048,34.285714,...,7.790476,10.759091,0.753664,14.275720,2.717300,65.454545,58.181818,40.625000,-0.086419,2005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1120,2BXRK001.64,2022-10-26,150,1.287,9.486085,0.952381,5.714286,0.952381,0.000000,13.333333,...,2.066667,2.736364,0.589700,4.640264,1.562427,67.090909,30.909091,26.470588,-1.157233,2022
1121,2-JKS070.06,2022-09-12,800,0.450,25.065800,9.523810,0.000000,0.000000,5.714286,30.476190,...,20.238095,29.218182,1.540245,18.969822,2.949336,27.636364,76.363636,12.380952,-0.052227,2022
1122,4ASNA007.82,2022-09-20,320,0.430,15.910317,0.000000,0.000000,0.000000,0.000000,0.000000,...,7.576190,13.836364,1.057536,13.083582,2.298445,95.909091,14.545455,82.500000,-1.817226,2022
1123,8-MIC001.47,2022-10-25,150,0.300,10.014865,0.000000,9.708738,0.000000,0.000000,0.000000,...,2.919048,4.390909,0.556709,7.887260,1.247618,89.090909,NaN,NaN,-1.544161,2022


In [18]:
# Push trimmed vadeq dataset to processed data folder
trimmed_path = os.path.join(BASE_DIR, "data", "processed", "trimmed_vdeq_emd.xlsx")
vadeq_emd.to_excel(trimmed_path)